# Tarefa 4: Executar o processamento de dados com o seu contêiner

No notebook anterior, você usou o Amazon SageMaker Processing e o contêiner integrado scikit-learn para o processamento de dados.

Neste notebook, você vai configurar o ambiente necessário para executar um script scikit-learn com o seu contêiner de processamento.

Você vai criar uma imagem do Docker, criar o contêiner de processamento e usar uma classe **ScriptProcessor** do SDK Python do SageMaker para executar um script scikit-learn de pré-processamento no contêiner.

Por fim, você vai validar os resultados do processamento de dados salvos no Amazon Simple Storage Service (Amazon S3).

## Tarefa 4.1: Configuração do ambiente

Nesta tarefa, você vai instalar as bibliotecas e as dependências necessárias. 

Você vai definir um bucket do Amazon S3 para armazenar as saídas do trabalho de processamento e também fazer com que a função de execução execute o trabalho do SageMaker Processing.

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

## Tarefa 4.2: Criar um contêiner de processamento

Nesta tarefa, você vai definir e criar um contêiner scikit-learn usando o Dockerfile.

### Tarefa 4.2.1: Criar um arquivo do Docker 

Nesta tarefa, você vai criar um diretório do Docker e adicionar o Dockerfile usado para criar o contêiner de processamento. Por estar criando um contêiner scikit-learn, você vai instalar o pandas e o scikit-learn.

In [ ]:
%mkdir docker

In [ ]:
%%writefile docker/Dockerfile
FROM public.ecr.aws/docker/library/python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

### Tarefa 4.2.2: Criar a imagem de contêiner

Nesta tarefa, você vai criar uma imagem de contêiner personalizada usando a Command Line Interface (CLI) de criação de imagem do Amazon SageMaker Studio. 

Usando a CLI de criação de imagem do Amazon SageMaker Studio, você pode criar imagens do Docker compatíveis com o Amazon SageMaker diretamente nos ambientes do SageMaker Studio.

Instale o pacote de criação de imagem do SageMaker Studio:

In [ ]:
%pip install sagemaker-studio-image-build

Navegue até o diretório que contém o Dockerfile e execute o comando de criação sm-docker. Esse comando registra em log automaticamente a saída do build e retorna o **URI da imagem** do Docker. O tempo de conclusão do comando é de cerca de dois minutos.

In [ ]:
%%sh

cd docker

sm-docker build .

Depois, copie o **URI da imagem** e cole-o em um editor de texto da sua escolha. 
Use esse **URI da imagem** para criar uma classe **ScriptProcessor**.

## Tarefa 4.3: Executar o trabalho de processamento do SageMaker

Nesta tarefa, você vai usar o mesmo conjunto de dados pré-processado do notebook anterior.

In [ ]:
#import-data
shape=pd.read_csv("data/adult_data.csv", header=None)
shape.sample(5)



Depois, você vai usar a classe ScriptProcessor do SageMaker para definir e executar um script de processamento como um trabalho de processamento. Consulte [SageMaker ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) para saber mais sobre essa classe.

Para criar a classe ScriptProcessor, configure os seguintes parâmetros:
- **base_job_name**: prefixo do nome do trabalho de processamento
- **command**: comando a ser executado, além dos sinalizadores da linha de comando
- **image_uri**: URI da imagem do Docker a ser usado para os trabalhos de processamento
- **role**: função de execução do SageMaker
- **instance_count**: número de instâncias para executar o trabalho de processamento
- **instance_type**: tipo de instância do Amazon Elastic Compute Cloud (Amazon EC2) usado para o trabalho de processamento

No código a seguir, substitua **REPLACE_IMAGE_URI** pelo URI do editor de texto.

In [ ]:
#sagemaker-script-processor
from sagemaker.processing import ScriptProcessor

# create a ScriptProcessor
script_processor = ScriptProcessor(
    base_job_name="own-processing-container",
    command=["python3"],
    image_uri="REPLACE_IMAGE_URI",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
)


Depois, use o método ScriptProcessor.run() para executar o script **sklearn_preprocessing.py** como um trabalho de processamento. Esse é o mesmo script que foi usado na Tarefa 3, mas agora ele está sendo executado em um contêiner personalizado criado por meio de uma imagem base. Consulte [ScriptProcessor.run()](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor.run) para saber mais sobre esse método.

Para executar o trabalho de processamento, configure os seguintes parâmetros:
- **code**: caminho do script de pré-processamento 
- **inputs**: caminho dos dados de entrada do script de pré-processamento (local de entrada do Amazon S3)
- **outputs**: caminho da saída do script de pré-processamento (local de saída do Amazon S3)
- **arguments**: argumentos de linha de comando para o script de pré-processamento (como a taxa de divisão do teste de treinamento)

O trabalho de processamento exige cerca de quatro a cinco minutos para ser concluído.

In [ ]:
#processing-job
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
script_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "adult_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                         source="/opt/ml/processing/test",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)
script_processor_job_description = script_processor.jobs[-1].describe()
print(script_processor_job_description)

## Tarefa 4.4: validar os resultados do processamento de dados

Valide a saída do trabalho de processamento que você executou consultando as cinco primeiras colunas dos conjuntos de dados de saída de treinamento e teste.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

### Conclusão

Parabéns! Você criou um contêiner de processamento e usou o SageMaker Processing para executar o trabalho de processamento com sucesso.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.